In [ ]:
##Message History

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}


def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model,get_session_history)


config = {"configurable":{"session_id":"chat1"}}



response = with_message_history.invoke(
    [HumanMessage(content="Hi, I am Siva and I am Developer")],
    config = config
)



In [ ]:
# with Prompt template

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt|model


with_message_history = RunnableWithMessageHistory(chain, get_session_history)


config3 = {"configurable":{"session_id":"chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content ="Hi my name is Guru")],
    config = config3
)



In [ ]:
## Mpre complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt|model



##n make more complicate
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = "messages")


config4 = {"configurable":{"session_id":"chat4"}}


response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, my name is sivaGuru ")], "language":"Tamil"},
    config = config4
    )

response.content

In [ ]:
## Manage Chat History  ### (trim_messages)

from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 75,
    strategy="last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)


messages = [
    SystemMessage(content="you're the good Assistant"),
    HumanMessage(content="hi, i'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="hi, I like venilla ice cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="What is 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problems!"),
    HumanMessage(content="Having fun!"),
    AIMessage(content="Yes!")
]



trimmer.invoke(messages)


from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)




with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config= {"configurable":{"session_id":"chat5"}}

response = with_message_history.invoke(
    {
    "messages" : messages + [HumanMessage(content="What is my name?")],
    "language" : "English"  
    },
    config=config
)


response.content
